In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=6ac40a7c0454d0ebabd26d3ee34a9df9864764ff5767273ce2de08720f4d3b05
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_2")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

# Самостоятельная работа к уроку 3
На уроке мы попробовали оконные и пользовательские функции. Теперь закрепим полученные знания.

## Данные: [google drive: raw_sales.csv](https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)

 Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):
*   date of sale
*   price
*   property type
*   number of bedrooms
*   4digit postcode

In [5]:
!wget 'https://drive.google.com/uc?export=download&id=1xbtFBPz50OBoyYFVGd-B03pCTJdCax07' -O raw_sales.csv

--2023-11-06 15:44:39--  https://drive.google.com/uc?export=download&id=1xbtFBPz50OBoyYFVGd-B03pCTJdCax07
Resolving drive.google.com (drive.google.com)... 173.194.216.101, 173.194.216.100, 173.194.216.139, ...
Connecting to drive.google.com (drive.google.com)|173.194.216.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-5k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jf5eqo0c52qr8qsomt86nmvefp6kldp4/1699285425000/04099736791713398091/*/1xbtFBPz50OBoyYFVGd-B03pCTJdCax07?e=download&uuid=9f38b248-6bc4-4113-a402-556334e29c5a [following]
--2023-11-06 15:44:39--  https://doc-0c-5k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jf5eqo0c52qr8qsomt86nmvefp6kldp4/1699285425000/04099736791713398091/*/1xbtFBPz50OBoyYFVGd-B03pCTJdCax07?e=download&uuid=9f38b248-6bc4-4113-a402-556334e29c5a
Resolving doc-0c-5k-docs.googleusercontent.com (doc-0c-5k-docs.googleusercontent.com)... 173.194.215.132, 

In [6]:
df = spark.read.csv('raw_sales.csv', header=True, inferSchema=True)
df.printSchema()
df.show(truncate=False)

root
 |-- datesold: timestamp (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- propertyType: string (nullable = true)
 |-- bedrooms: integer (nullable = true)

+-------------------+--------+-------+------------+--------+
|datesold           |postcode|price  |propertyType|bedrooms|
+-------------------+--------+-------+------------+--------+
|2007-02-07 00:00:00|2607    |525000 |house       |4       |
|2007-02-27 00:00:00|2906    |290000 |house       |3       |
|2007-03-07 00:00:00|2905    |328000 |house       |3       |
|2007-03-09 00:00:00|2905    |380000 |house       |4       |
|2007-03-21 00:00:00|2906    |310000 |house       |3       |
|2007-04-04 00:00:00|2905    |465000 |house       |4       |
|2007-04-24 00:00:00|2607    |399000 |house       |3       |
|2007-04-30 00:00:00|2606    |1530000|house       |4       |
|2007-05-24 00:00:00|2902    |359000 |house       |3       |
|2007-05-25 00:00:00|2906    |320000 |house       |3   

## Задание 1
Добавьте к таблице следующие поля:
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего (1 балл)


In [7]:
from pyspark.sql import Window
import pyspark.sql.functions as F

# Средняя стомость 10 проданных домов до текущего в том же районе (в SQL запросе)
df.registerTempTable('df_task_1_1')

spark.sql('''
select *,
AVG(price) OVER(PARTITION BY postcode ORDER BY datesold ROWS BETWEEN 9 PRECEDING AND CURRENT ROW) AS avg_10_prev
FROM df_task_1_1
''').show(15)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+-------------------+--------+-------+------------+--------+-----------------+
|           datesold|postcode|  price|propertyType|bedrooms|      avg_10_prev|
+-------------------+--------+-------+------------+--------+-----------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|         327000.0|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|         558500.0|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|647333.3333333334|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|         564250.0|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|         509900.0|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|         479750.0|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|         520500.0|
|2008-07-29 00:00:00|    2600| 927000|       house|       4|         571312.5|
|2008-09-02 00:00:00|    2600|1380000|       house|       5|661166.6666666666|
|2008-09-08 00:00:00|    2600| 740000|       house| 

## Задание 2
Найдите среднюю цену жилья для каждого года и приджойните эти данные к таблице из задания 1. (2 балла)


*(left join on a.year(date of sale) = b.year, где a - таблица из первого задания, а b таблица после группировки)*

In [8]:
# Средняя стомость 10 проданных домов до текущего в том же районе (в pyspark)
w = Window().partitionBy('postcode')\
            .orderBy('datesold')\
            .rowsBetween(Window.currentRow - 9, Window.currentRow)

df.withColumn('avg_10_prev', F.mean('price').over(w)).show(15)

+-------------------+--------+-------+------------+--------+-----------------+
|           datesold|postcode|  price|propertyType|bedrooms|      avg_10_prev|
+-------------------+--------+-------+------------+--------+-----------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|         327000.0|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|         558500.0|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|647333.3333333334|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|         564250.0|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|         509900.0|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|         479750.0|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|         520500.0|
|2008-07-29 00:00:00|    2600| 927000|       house|       4|         571312.5|
|2008-09-02 00:00:00|    2600|1380000|       house|       5|661166.6666666666|
|2008-09-08 00:00:00|    2600| 740000|       house| 

In [9]:
# Средняя стомость 10 проданных домов после текущего в том же районе (в SQL)
df.registerTempTable('df_task_1_2')

spark.sql('''
select *,
AVG(price) OVER(PARTITION BY postcode ORDER BY datesold ROWS BETWEEN CURRENT ROW AND 9 FOLLOWING) AS avg_10_next
FROM df_task_1_2
''').show(15)

+-------------------+--------+-------+------------+--------+-----------+
|           datesold|postcode|  price|propertyType|bedrooms|avg_10_next|
+-------------------+--------+-------+------------+--------+-----------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|   669050.0|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|   708350.0|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|   698350.0|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|   679350.0|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|   742850.0|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|   786600.0|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|   839200.0|
|2008-07-29 00:00:00|    2600| 927000|       house|       4|   868450.0|
|2008-09-02 00:00:00|    2600|1380000|       house|       5|   805750.0|
|2008-09-08 00:00:00|    2600| 740000|       house|       3|   715250.0|
|2008-09-17 00:00:00|    2600| 720000|       house|

In [10]:
# Стоимость последнего проданного дома до текущего (в SQL)
df.registerTempTable('df_task_1_3')

spark.sql('''
select *,
LAG(price, 1, 0) OVER(ORDER BY datesold) AS prev_price
FROM df_task_1_3
''').show(15)

+-------------------+--------+-------+------------+--------+----------+
|           datesold|postcode|  price|propertyType|bedrooms|prev_price|
+-------------------+--------+-------+------------+--------+----------+
|2007-02-07 00:00:00|    2607| 525000|       house|       4|         0|
|2007-02-27 00:00:00|    2906| 290000|       house|       3|    525000|
|2007-03-07 00:00:00|    2905| 328000|       house|       3|    290000|
|2007-03-09 00:00:00|    2905| 380000|       house|       4|    328000|
|2007-03-21 00:00:00|    2906| 310000|       house|       3|    380000|
|2007-04-04 00:00:00|    2905| 465000|       house|       4|    310000|
|2007-04-24 00:00:00|    2607| 399000|       house|       3|    465000|
|2007-04-30 00:00:00|    2606|1530000|       house|       4|    399000|
|2007-05-24 00:00:00|    2902| 359000|       house|       3|   1530000|
|2007-05-25 00:00:00|    2906| 320000|       house|       3|    359000|
|2007-06-26 00:00:00|    2902| 385000|       house|       3|    

In [11]:
# Стоимость последнего проданного дома до текущего (в pyspark)

w = Window().orderBy('datesold')
df.withColumn('prev_price', F.lag('price', 1, 0).over(w)).show(15)

+-------------------+--------+-------+------------+--------+----------+
|           datesold|postcode|  price|propertyType|bedrooms|prev_price|
+-------------------+--------+-------+------------+--------+----------+
|2007-02-07 00:00:00|    2607| 525000|       house|       4|         0|
|2007-02-27 00:00:00|    2906| 290000|       house|       3|    525000|
|2007-03-07 00:00:00|    2905| 328000|       house|       3|    290000|
|2007-03-09 00:00:00|    2905| 380000|       house|       4|    328000|
|2007-03-21 00:00:00|    2906| 310000|       house|       3|    380000|
|2007-04-04 00:00:00|    2905| 465000|       house|       4|    310000|
|2007-04-24 00:00:00|    2607| 399000|       house|       3|    465000|
|2007-04-30 00:00:00|    2606|1530000|       house|       4|    399000|
|2007-05-24 00:00:00|    2902| 359000|       house|       3|   1530000|
|2007-05-25 00:00:00|    2906| 320000|       house|       3|    359000|
|2007-06-26 00:00:00|    2902| 385000|       house|       3|    

## Задание 3
В итоге у вас таблица с колонками (или нечто похожее):
*   price
*   Среднегодовая цена
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего ((1 балл)
*  и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)). (2 балла)

In [12]:
w_y = Window().orderBy('year')

w_prev = Window().partitionBy('postcode')\
            .orderBy('datesold')\
            .rowsBetween(Window.currentRow - 9, Window.currentRow)

w_next = Window().partitionBy('postcode')\
            .orderBy('datesold')\
            .rowsBetween(Window.currentRow, Window.currentRow + 9)

w = Window().orderBy('datesold')

df = df.withColumn('year', F.split("datesold", "-").getItem(0))\
       .withColumn('mean', F.mean('price').over(w_y))\
       .withColumn('avg_10_prev', F.mean('price').over(w_prev))\
       .withColumn('avg_10_next', F.mean('price').over(w_next))\
       .withColumn('prev_price', F.lag('price', 1, 0).over(w))

df.show(15)

+-------------------+--------+-------+------------+--------+----+-----------------+-----------------+-----------+----------+
|           datesold|postcode|  price|propertyType|bedrooms|year|             mean|      avg_10_prev|avg_10_next|prev_price|
+-------------------+--------+-------+------------+--------+----+-----------------+-----------------+-----------+----------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|2007|522377.2108843537|         327000.0|   669050.0|    900000|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|2007|522377.2108843537|         558500.0|   708350.0|    625000|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|2007|522377.2108843537|647333.3333333334|   698350.0|    620000|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|2008| 499156.106870229|         564250.0|   679350.0|    580000|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|2008| 499156.106870229|         509900.0|   742850.0|    445000|


In [13]:
l = lambda row: '**'.join(row) # объединим все ячейки
l_udf = F.udf(l, returnType='string')

df.select(l_udf(F.array_distinct(F.array('*')))).count() # посчитаем кол-во уникальных

29580

In [14]:
l = lambda row: '**'.join(row) # объединим все ячейки
l_udf = F.pandas_udf(l, returnType='string')

df.select(l_udf(F.array_distinct(F.array('*')))).count() # посчитаем кол-во уникальных

29580